In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Vengono importati i dati che verrannno utilizzati per testare i metodi.

In [ ]:
Mall_Customers=pd.read_csv('../input/mall-customers/Mall_Customers.csv')
Iris = pd.read_csv("../input/iris/Iris.csv")
Seed_Data = pd.read_csv("../input/seed-from-uci/Seed_Data.csv")
X_C1=[]
with open('/kaggle/input/uefclusters2/s1.txt') as f:
    for line in f:
        x, y = line.split()
        X_C1.append([int(x),int(y)])
X_C2=[]
with open('/kaggle/input/uefclusters2/s2.txt') as f:
    for line in f:
        x, y = line.split()
        X_C2.append([int(x),int(y)])
X_C3=[]
with open('/kaggle/input/uefclusters2/s3.txt') as f:
    for line in f:
        x, y = line.split()
        X_C3.append([int(x),int(y)])
X_C4=[]
with open('/kaggle/input/uefclusters2/s4.txt') as f:
    for line in f:
        x, y = line.split()
        X_C4.append([int(x),int(y)])

X_M=Mall_Customers.values[:,(3,4)]
X_I=Iris.values[:,(1,2,3,4)]
X_S=Seed_Data.values[:,range(0,7)]

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


Verranno utilizzate due metriche per valutare il numero di cluster da utilizzare: l'inerzia (iner), che è la somma del quadrato delle distanze dal centroide più vicino, e la silhouette, che è data da (b - a)/max(a, b), dove a è la distanza media all'interno di un cluster e b è la distanza media tra cluster. Si calcola il valore di queste due metriche tramite la funzione metricGenerator.

In [ ]:
def metricGenerator(X):
    sil=[0,0]
    iner=[]
    N=30
    for k in range(1, N):
            km = KMeans(init="k-means++", n_clusters=k, n_init=20)
            km.fit(X)
            iner.append(km.inertia_)
            if (k>1): 
                sil.append(silhouette_score(X,km.labels_))
    n=iner[0]
    for k in range(0,N-1):
        iner[k] = iner[k]/n
    return ([iner,sil])

Si possono osservare quindi i grafici di queste metriche applicati al dateset Mall_Customers:

In [ ]:
mall=metricGenerator(X_M)
import pylab as pl
pl.plot(mall[0])
pl.xlabel("numero di cluster")
pl.ylabel("inerzia")
pl.grid()
pl.show()

pl.plot(mall[1])
pl.grid()
pl.xlabel("numero di cluster")
pl.ylabel("silhouette score")
pl.show()

Mentre per il grafico della silhouette è sufficente identificare il massimo per trovare il numero ottimale di cluster, l'inerzia è un valore che diminuisce con il numero di cluster usati, e si utilizza il metodo del gomito, spiegato in [questo blog](https://www.developersmaggioli.it/blog/apprendimento-non-supervisionato-clustering-k-means/). Bisogna quindi identificare il punto di gomito. Se si assume che dopo il punto di gomito il comportamento dell'inerzia è approssimativamente lineare, lo si può trovare usando la versione discreta della derivata seconda (che sarà nulla per una funzione lineare) ossia il rapporto incrementale del rapporto incrementale:

In [ ]:
def dDer(lis):
    ret=[]
    for k in range (0,len(lis)-1):
        ret.append(lis[k]-lis[k+1])
    return(ret)

Il grafico seguente mostra la derivata dell'inerzia del dataset utilizzato.

In [ ]:
pl.plot(dDer(dDer(mall[0])))
pl.xlabel("numero di cluster")
pl.ylabel("derivata seconda")
pl.grid()
pl.show()

Il gomito sarà il punto dove la derivata seconda diventa (approssimativamente) nulla, quindi un gomito sarà identificato quando la media dei quadrati di tre valori successivi sarà minore di una solgia data. Si usa la media di tre valori per evitare che un singolo punto, dove la derivata seconda è nulla, venga erroenamente identificato come punto di gomito, come avviene nel grafico soprastante per 2. La soglia verrà impostata d'ora in poi come 10^-4. Si può usare questo valore costante in quanto la funzione metricGenerator normalizza l'inerzia. Ci si aspetta, quindi, una soglia ad un valore simile, ma si potrebbe cercare un criterio migliore per la scelta di questo iperparametro. Non sarà però necessario in quanto verrà mostrato successivamente un metodo migliore.

In [ ]:
def findElbow(lis,slg):
        dd=dDer(dDer(lis))
        for k in range(0,len(dd)-1):
            if (dd[k]**2+dd[k+1]**2+dd[k+2]**2)/3<slg: 
                return(k)
        return(len(dd)-1)
            

Vengono applicati questi metodi al dataset dato, tenendo conto del fatto che il gomito trovato è più piccolo di 1 del valore cercato in quanto l'indice dell'array inizia a 0 ma l'inerzia viene calcolata per un numero di cluster maggiore uguale a 1. 

In [ ]:
print("silhouette:",mall[1].index(max(mall[1]))," gomito:",findElbow(mall[0],10**(-4))+1)

I due metodi concordano su 5 come numero di cluster. Segue il grafico dei dati divisi in 5 cluster:

In [ ]:
pl.scatter(X_M[:,0],X_M[:,1],c=KMeans(init="k-means++", n_clusters=5, n_init=20).fit(X_M).labels_)
pl.xlabel("annual income")
pl.ylabel("spending score")
pl.show()

Si può quindi vedere che i metodi hanno entrambi identificato il numero corretto di cluster.
Applicando però questo metodo ad altri dataset si possono mettere in evidenza dei problemi. Viene studiato il dataset s1 preso da http://cs.joensuu.fi/sipu/datasets/ di cui è noto il numero di cluster da cui è composto (15).

In [ ]:
S1=metricGenerator(X_C1)

In [ ]:
pl.plot(S1[0])
pl.xlabel("numero di cluster")
pl.ylabel("inerzia")
pl.grid()
pl.show()
pl.plot(dDer(dDer(S1[0])))
pl.xlabel("numero di cluster")
pl.ylabel("derivata seconda dell'inerzia")
pl.grid()
pl.show()
print(" gomito:",findElbow(S1[0],10**(-4))+1)

Si può vedere dal grafico della derivata seconda che ci sono due gomiti (due sezioni diverse dove la fuznione è lineare), e che l'algoritmo identifica il gomito errato. Questo problema potrebbe essere risolto modificando la soglia, ma in questo modo fallirebbe su altri dataset.
La soluzione proposta richiede una funzione che identifica più gomiti presenti, e seleziona il "migliore". Come gomito migliore viene selezionato quello dove il rapporto tra i valori della derivata seconda prima e dopo il gomito è maggiore, in questo modo la performance migliora sensibilmente.

In [ ]:
def findElbowCandidates(lis,slg,slg2):
    dd=dDer(dDer(lis))
    el=[]
    f=slg2
    for k in range(0,len(dd)-2):
        if f<slg2:
            f=(dd[k]**2+dd[k+1]**2+dd[k+2]**2)/3
        else:
            if (dd[k]**2+dd[k+1]**2+dd[k+2]**2)/3<slg: 
                el.append([k,(dd[k]**2+dd[k+1]**2+dd[k+2]**2)/3,f])
            f=(dd[k]**2+dd[k+1]**2+dd[k+2]**2)/3
    return(el)
    

In [ ]:
def findBestElbow(lis,slg,slg2):
    el=findElbowCandidates(lis,slg,slg2)
    el_v=[]
    for k in range(0,len(el)):
        el_v.append(el[k][2]/el[k][1])
    return(el[el_v.index(max(el_v))][0])


Applicando i metodi fin'ora elencati al dataset S1 si ottengono quindi i seguenti risultati:


In [ ]:
print("silhouette:",S1[1].index(max(S1[1]))," gomito semplice:",findElbow(S1[0],10**(-4))+1,  "gomito migliorato:", findBestElbow(S1[0],10**(-3),10**(-6))+1)

Verificato che il nuovo metodo trovato ci restituisce il valore voluto si può procedere a testare su altri dataset.
La funzione creata di seguito bestClusterNumber restituisce sotto forma di array il numero di cluster predetto da findBestElbow, dalla silhouette, dalla prima versione del metodo del gomito, dal valore massimo degli ultimi due metodi e dal valore massimo di tutti e tre i metodi. Tramite questa operazione di "pooling" delle varie predizioni si ottiene una predizione migliore delle singole predizioni. Si sceglie il valore maggiore in quanto, come vedremo successivamente, i metodi selezionati approssimano quasi esclusivamente per difetto il numero di cluster.


In [ ]:
def bestClusterNumber(lis):
    be = findBestElbow(lis[0],10**(-3),10**(-6))+1
    si = lis[1].index(max(lis[1]))
    se = findElbow(lis[0],10**(-4))+1
    return([be,si,se,max(se,si),max(se,si,be)])

In [ ]:
bestClusterNumber(S1)

In [ ]:
def printBCN(name,target,res):
    print("dataset",name,"formato da",target,"clusters:      gomito migliorato:", res[0]," silhouette:",res[1]," gomito semplice:",res[2], " pool2:",res[3]," pool3:",res[4] )
def findBCN(name,target,data):
    printBCN(name,target,bestClusterNumber(metricGenerator(data)))

Applicando i metodi ai 3 dataset reali e ai 4 dataset generati si ottiengono i seguenti risultati:

In [ ]:
findBCN("Mall_Customers",5, X_M)
findBCN("Iris Species",3, X_I)
findBCN("Seed_from_UCI",3, X_S)
findBCN("S1",15, X_C1)
findBCN("S2",15, X_C2)
findBCN("S3",15, X_C3)
findBCN("S4",15, X_C4)

Si possono ulteriormente testare i metodi usati su una serie di dataset generati artificialmente tramite la funzione di sklearn make_blobs, che genererà dataset simili al seguente, avente come parametri 4 cluster e deviazione standard pari a 0.8:

In [ ]:
from sklearn.datasets import make_blobs
X, Y= make_blobs(n_samples=500, centers=4, cluster_std=0.8, random_state=0)
pl.scatter(X[:,0],X[:,1],c=Y)
pl.show()

Il test verrà fatto su un numero di cluster che va da 3 a 8, questi generati con deviazione stardand da 0.4 a 1.2 con incrementi di 0.2. Per ogni configurazione verranno generati 4 dataset diversi.

In [ ]:

rand_counter=0
ind=[]
per=[]
for cl_n in range(3,9):
    ind2=[]
    per2=[]
    for std in range (2,7):
        ind1=[]
        per1=[]
        for rand in range(1,5):
            
            X, Y= make_blobs(n_samples=500, centers=cl_n, cluster_std=std/5, random_state=rand_counter)
            Z=bestClusterNumber(metricGenerator(X))
            ind1.append(Z)
            per1.append(list(map(lambda x:x-cl_n, Z)))
            #print(rand,std,cl_n," -> ", Z)
            rand_counter+=1
        ind2.append(ind1)
        per2.append(per1)
    ind.append(ind2)
    per.append(per2)
ind=np.asarray(ind)
per=np.asarray(per)
print(per)

I risultati del test sono organizzati in due matrici 6x5x4x5, che corrisponde a numero di cluster x deviazione standard x iterazione x metodo usato. In ind ogni elemento della matrice indica la predizione di numero di cluster per un dato dataset tramite un dato metodo, mentre la matrice per indica la differenza tra questo valore e la predizione corretta. Si userà quest'ultimo valore per valutare la performance dei metodi usati tramite due metriche: errore medio, che indica la differenza in media tra numero di cluster predetto e numero di cluster corretto, e hit rate, ossia il rapporto tra il numero di predizioni corrette e numero di predizioni totali. Viene calcolata anche quante volte la stima è avvenuta per eccesso, per confermare la validità del pooling tramite scelta del valore massimo.

In [ ]:
def avErr (lis):
    return(sum(map(abs,lis))/len(lis))
def avMissErr(lis):
    return(sum(map(abs,lis))/sum(map(lambda x:x!=0,lis)))
def avHitR (lis):
    return(sum(map(lambda x:x==0,lis))/len(lis))
def totOv (lis):
    return(sum(map(lambda x:x>0,lis))/len(lis))
def flt(lis):
    return([y for x in lis for y in x])
def flt2(lis):
    return(flt(flt(lis)))
def printscore(name,lis):
    print("Performance",name,": Errore medio:",avErr(lis),"Predizioni corrette:", avHitR(lis),"Errore medio sulle predizoni errate:",avMissErr(lis) , "Stime per ecesso:", totOv(lis) )
def score(lis):
    printscore("gomito migliorato",flt2(lis[:,:,:,0]))
    printscore("silhouette",flt2(lis[:,:,:,1]))
    printscore("gomito semplice",flt2(lis[:,:,:,2]))
    printscore("pooling di 2",flt2(lis[:,:,:,3]))
    printscore("pooling di 3",flt2(lis[:,:,:,4]))

In [ ]:
score(per)

In conclusione, si può quindi vedere come il metodo del gomito migliorato performa meglio sia degli altri due metodi sia del loro pooling, e come in ogni caso il pooling di più metodi migliora la peformance globale.